# Introduction 

The objective of this session is to implement a convolutional network and test the influence of the architecture on the performance.

# 1 Training function

Re-organize the code to define and use a function
train model(model, train input, train target, mini batch size)
Hint: My version is 605 characters long

In [1]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

train_input, train_target, test_input, test_target = \
    prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [3]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        nb_hidden = 200
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.fc1 = nn.Linear(9 * 64, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x.view(-1, 9 * 64)))
        x = self.fc2(x)
        return x

###  Correction

In [4]:
def train_model(model, train_input, train_targe, mini_batch_size):
    criterion = nn.MSELoss()
    eta = 1e-1

    for e in range(0, 25):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.data[0]
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print(e, sum_loss)

# 2 Test error

Write and test a function
    
    compute nb errors(model, input, target, mini batch size)

To compute the number of prediction mistakes using a “winner-take-all” rule, that is the class with the largest output is the predicted one.

Run the training and test ten times, record the test error rates.
With 25 epochs for training, the test error should be around 10% with the small sets, and around
0.7% with the full ones.

Hint: My version is 424 characters long

### Correction

In [11]:
def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0
    
    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.data.max(1)
        for k in range(0, mini_batch_size):
            if target.data[b + k, predicted_classes[k]] < 0:
                nb_errors = nb_errors + 1

    return nb_errors

In [12]:
train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

mini_batch_size = 100

In [13]:
for k in range(0, 10):
    model = Net(200)
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))



/Users/amarchand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if sys.path[0] == '':


0 tensor(5.1537)
1 tensor(3.8997)
2 tensor(3.4844)
3 tensor(3.0804)
4 tensor(2.7258)
5 tensor(2.5594)
6 tensor(2.2573)
7 tensor(2.1074)
8 tensor(1.9079)
9 tensor(1.5862)
10 tensor(1.7229)
11 tensor(1.5882)
12 tensor(1.3603)
13 tensor(1.2518)
14 tensor(1.2755)
15 tensor(1.2214)
16 tensor(1.1897)
17 tensor(1.0081)
18 tensor(1.0524)
19 tensor(0.9699)
20 tensor(0.9247)
21 tensor(0.8529)
22 tensor(0.8888)
23 tensor(0.8628)
24 tensor(0.8007)
test error Net 11.80%% 118/1000
0 tensor(5.1188)
1 tensor(3.7244)
2 tensor(3.2644)
3 tensor(2.8402)
4 tensor(2.6329)
5 tensor(2.2947)
6 tensor(2.1035)
7 tensor(2.0711)
8 tensor(1.8872)
9 tensor(1.7262)
10 tensor(1.6084)
11 tensor(1.4502)
12 tensor(1.2982)
13 tensor(1.2662)
14 tensor(1.2135)
15 tensor(1.1865)
16 tensor(1.1038)
17 tensor(0.9941)
18 tensor(1.0208)
19 tensor(0.9709)
20 tensor(0.9136)
21 tensor(0.8325)
22 tensor(0.8281)
23 tensor(0.7610)
24 tensor(0.7430)
test error Net 8.80%% 88/1000
0 tensor(5.4364)
1 tensor(4.0331)
2 tensor(3.3610)
3 tenso

# 3 Influence of the number of hidden units

In the default network, the number of hidden units is 200.

Modify the class constructor to take a parameter for that value, and run the training and compute the
test error for 10, 50, 200, 500, and 1, 000 hidden units.

### Correction

In [14]:
for nh in [ 10, 50, 200, 500, 2500 ]:
    model = Net(nh)
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net nh={:d} {:0.2f}%% {:d}/{:d}'.format(nh,
                                                              (100 * nb_test_errors) / test_input.size(0),
                                                              nb_test_errors, test_input.size(0)))

/Users/amarchand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if sys.path[0] == '':


0 tensor(7.8041)
1 tensor(4.3644)
2 tensor(4.0101)
3 tensor(3.8391)
4 tensor(3.7193)
5 tensor(3.6325)
6 tensor(3.5630)
7 tensor(3.4914)
8 tensor(3.4466)
9 tensor(3.3581)
10 tensor(3.3357)
11 tensor(3.3114)
12 tensor(3.2417)
13 tensor(3.2636)
14 tensor(3.1930)
15 tensor(3.0752)
16 tensor(3.1080)
17 tensor(3.0855)
18 tensor(3.2438)
19 tensor(3.0729)
20 tensor(3.0196)
21 tensor(2.9701)
22 tensor(3.0001)
23 tensor(2.9702)
24 tensor(2.8910)
test error Net nh=10 60.80%% 608/1000
0 tensor(5.4306)
1 tensor(4.0868)
2 tensor(3.6815)
3 tensor(3.3588)
4 tensor(3.0558)
5 tensor(3.0086)
6 tensor(2.8910)
7 tensor(2.5622)
8 tensor(2.5182)
9 tensor(2.1657)
10 tensor(1.9295)
11 tensor(2.0259)
12 tensor(1.9314)
13 tensor(1.7201)
14 tensor(1.6274)
15 tensor(1.6221)
16 tensor(1.4605)
17 tensor(1.4907)
18 tensor(1.3746)
19 tensor(1.3761)
20 tensor(1.2511)
21 tensor(1.2418)
22 tensor(1.1342)
23 tensor(1.1547)
24 tensor(1.0695)
test error Net nh=50 13.90%% 139/1000
0 tensor(4.4659)
1 tensor(3.6202)
2 tensor(3

# 4 Three convolutional layers

Write a new class Net2 with three convolutional layers. 
Pick the structure you want.

### Correction

In [15]:
model = Net2()
train_model(model, train_input, train_target, mini_batch_size)
nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
print('test error Net2 {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                   nb_test_errors, test_input.size(0)))

/Users/amarchand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if sys.path[0] == '':


0 tensor(6.2000)
1 tensor(4.2079)
2 tensor(3.6673)
3 tensor(3.4525)
4 tensor(3.1880)
5 tensor(2.8472)
6 tensor(2.6063)
7 tensor(2.3734)
8 tensor(2.1584)
9 tensor(2.0831)
10 tensor(1.9571)
11 tensor(1.7990)
12 tensor(1.6794)
13 tensor(1.4944)
14 tensor(1.5140)
15 tensor(1.3356)
16 tensor(1.3059)
17 tensor(1.2490)
18 tensor(1.1908)
19 tensor(1.0492)
20 tensor(1.0883)
21 tensor(0.9918)
22 tensor(0.9318)
23 tensor(0.8729)
24 tensor(0.9173)
test error Net2 10.50%% 105/1000
